In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train (1).csv


In [0]:

import numpy as np
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
import timeit
import re
stemmer = nltk.stem.snowball.SnowballStemmer('english')

In [0]:
import io
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [4]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test (1).csv


In [0]:
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [12]:
test["comment_text"][20266] = "nigger radical nigg"
test["comment_text"][111567] = "boner orgasm gasm"
train["comment_text"][56313] = "thanks for your concern"
train["comment_text"][128782] = "fuck sex"
train["comment_text"][40223] = "SPANISH CENTRAL IS STUPID"
test["comment_text"][47590] = "nguyen do is a cock sucker"
train["comment_text"][128782] = "fuck sex"

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages

In [0]:
def cleaned(comment):
    comment=comment.lower()
    comment=re.sub("\\n"," ",comment)
    comment=re.sub("\d{1,}","",comment)
    comment=re.sub("\.{1,}",".",comment)
    comment=re.sub("\:{1,}","",comment)
    comment=re.sub("\;|\=|\%|\^|\_"," ",comment)
    comment=re.sub("\""," ",comment)
    comment=re.sub("\'{2,}","",comment)
    comment=re.sub("\/|\!"," ",comment)
    comment=re.sub("\?"," ",comment)
    comment=re.sub("\#"," ",comment)
    comment=re.sub("\,|\@|\|"," ",comment)
    comment=re.sub("\(|\)"," ",comment)
    comment=re.sub("\S+jpg"," ",comment)
    comment=re.sub("\S*wikip\S+","",comment)               
    comment=re.sub("\[.*?\]"," ",comment)
    comment=re.sub("\-"," ",comment)
    '''comment=re.sub("\"|:|@|,|\/|\=|;|\.|\'|\?|\!|\||\+|\~|\-|\#"," ",comment)
    comment=re.sub("\.{1,}",".",comment)
    comment=re.sub("\[.*?\]","",comment)
    comment=re.sub("www\S+","",comment)
    comment=re.sub("\_"," ",comment)
    comment=re.sub("http","",comment)'''
    comment=re.sub(r'[^\x00-\x7F]+',' ', comment) # remove non ascii
    comment=re.sub("\s+"," ",comment)
    comment = ' '.join( [w for w in comment.split() if len(w)>1])
    comment = ' '.join( [stemmer.stem(w) for w in comment.split()])
    comment = comment.strip()
    return comment

In [0]:
train['comment_text']=train['comment_text'].apply(lambda x :cleaned(x))
test['comment_text']=test['comment_text'].apply(lambda x :cleaned(x))

In [0]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [0]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=5000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)
#audio()

char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 3),
    max_features=5000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [0]:
train_features = hstack([train_char_features, train_word_features])
test_features = hstack([test_char_features, test_word_features])

scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})

In [18]:
for class_name in class_names:
    if (class_name in ['toxic']):
        train_target = train[class_name]
        classifier = LogisticRegression(C=0.63, solver='sag', class_weight = "balanced") # sag arge datasets and bivariate
        cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
        scores.append(cv_score)
        print('CV score for class {} is {}'.format(class_name, cv_score))
        classifier.fit(train_features, train_target)
        submission[class_name] = classifier.predict_proba(test_features)[:, 1]
    
    elif(class_name in ["severe_toxic", "insult"]):
        train_target = train[class_name]
        classifier = LogisticRegression(C=0.38, solver='sag') # sag large datasets and bivariate
        cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
        scores.append(cv_score)
        print('CV score for class {} is {}'.format(class_name, cv_score))
        classifier.fit(train_features, train_target)
        submission[class_name] = classifier.predict_proba(test_features)[:, 1]
        
    elif(class_name in ["threat", "identity_hate"]):
        train_target = train[class_name]
        classifier = LogisticRegression(C=0.45, solver='sag') # sag large datasets and bivariate
        cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
        scores.append(cv_score)
        print('CV score for class {} is {}'.format(class_name, cv_score))
        classifier.fit(train_features, train_target)
        submission[class_name] = classifier.predict_proba(test_features)[:, 1]
        
    elif(class_name == "obscene"):
        print("OBSCENE class")
        train_target = train[class_name]
        classifier = Ridge(alpha=20, solver='auto',max_iter=100, random_state=22,  tol=0.0005)
        cv_score = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
        scores.append(cv_score)
        print('CV score for class {} is {}'.format(class_name, cv_score))
        classifier.fit(train_features, train_target)
        submission[class_name] = classifier.predict(test_features)

print('Total CV score is {}'.format(np.mean(scores)))

CV score for class toxic is 0.9759697439845137
CV score for class severe_toxic is 0.988513233032757
OBSCENE class
CV score for class obscene is 0.9919301761363307
CV score for class threat is 0.9865372456834413
CV score for class insult is 0.9807187976761984
CV score for class identity_hate is 0.981153125633155
Total CV score is 0.9841370536910662
